In [1]:
import sys
sys.path.insert(0, '../sbident')

import numpy as np
from sbident import SBIdent
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons
from astropy import units as u

# sbident examples

0. [Call the API with the minimum required arguments: observer location, time and field of view](#Simplest-call)
1. [Expand on the simple example to include a magnitude limit, a field of view width constraint](#A-more-typical-simple-case)
2. [Use a spacecraft as the observer location via Horizons](#Use-xobs-for-a-spacecraft-observer-location)
3. [Have the API return orbital elements rather than RA/DEC](#Request-orbital-elements-instead-of-RA/DEC)
4. [Defined the field of view via a boundary rather than center + half width](#Use-an-edges-based-field-of-view-instead-of-center-based)

### Simplest call

Call the JPL Small Body Identification API for an observer at MPC observatory code **567** looking at the sky centered on **center** at the observation **time**.

In [2]:
#setup some arguments for the call
center = SkyCoord(10, -20, unit="deg")
time = Time('2021-10-01 00:00:00')

In [3]:
#call the API
sbid0 = SBIdent('567', time, center)

#NOTE: the line below is an identical call invoked with keyword arguments
#sbid0 = SBIdent(location='567', obstime=time, location=center)

In [4]:
#number of results in table
len(sbid0.results)

12

In [5]:
#first 5 results
sbid0.results[0:5]

Object name,Astrometric RA (hh:mm:ss),"Astrometric Dec (dd mm'ss"")","Dist. from center RA ("")","Dist. from center Dec ("")","Dist. from center Norm ("")",Visual magnitude (V),"RA rate (""/h)","Dec rate (""/h)"
str19,str11,str12,str5,str5,str5,str4,str10,str10
149544 (2003 HY50),00:40:27.54,"-20 08'03.1""",388.,-483.,620.9,19.0,-2.883E+01,-7.943E+00
172806 (2004 GT10),00:40:33.40,"-19 43'58.6""",471.,961.,1071.,18.7,-2.659E+01,-1.926E+01
225866 (2001 XM223),00:39:16.59,"-19 52'34.6""",-612.,445.,757.0,19.0,-3.871E+01,5.329E+00
255370 (2005 WS111),00:41:44.59,"-20 05'42.9""",1.E3,-343.,1512.,18.8,-3.015E+01,-5.458E+00
272760 (2005 YG152),00:38:34.58,"-19 57'30.9""",-1.E3,149.,1213.,20.2,-3.083E+01,-1.330E+01


### Add a magnitude constraint and expand the field of view

Expand on the previous call.  This time limit objects returned to **19th magnitude** or less; expand the field of view **half-width to 1 degree** from the default 0.5.

In [6]:
sbid1 = SBIdent('567', time, center, maglim=19, hwidth=1)

In [7]:
len(sbid1.results)

7

In [8]:
sbid1.results 

Object name,Astrometric RA (hh:mm:ss),"Astrometric Dec (dd mm'ss"")","Dist. from center RA ("")","Dist. from center Dec ("")","Dist. from center Norm ("")",Visual magnitude (V),"RA rate (""/h)","Dec rate (""/h)"
str19,str11,str12,str5,str5,str5,str4,str10,str10
35889 (1999 JA81),00:41:42.21,"-19 04'47.8""",1.E3,3.E3,3614.,18.6,-3.174E+01,-1.306E+01
57855 (2001 XT144),00:40:35.35,"-20 39'18.8""",498.,-2.E3,2411.,18.8,-3.346E+01,-9.137E+00
91101 (1998 HK15),00:37:26.23,"-20 33'44.7""",-2.E3,-2.E3,2963.,17.7,-3.600E+01,-6.281E-01
93867 (2000 WQ115),00:41:43.02,"-19 05'14.9""",1.E3,3.E3,3593.,17.8,-3.701E+01,-8.077E-01
120306 (2004 KE6),00:36:31.25,"-20 30'11.0""",-3.E3,-2.E3,3451.,19.0,-3.658E+01,-2.018E+00
172806 (2004 GT10),00:40:33.40,"-19 43'58.6""",471.,961.,1071.,18.7,-2.659E+01,-1.926E+01
255370 (2005 WS111),00:41:44.59,"-20 05'42.9""",1.E3,-343.,1512.,18.8,-3.015E+01,-5.458E+00


### Use xobs for a spacecraft observer location

Using a Geocentric state vector and the **xobs** key with your location argument you can request data from the perspective of an arbitrary observer.  You can also use **xobs-hel** to specify an observer from a Heliocentric perspective.

In [9]:
#get state of TESS (-95) from Horizons at our observation time
#and convert it from [AU, AU/day] to [km, km/s]

au = (1*u.au).to(u.km).value #1AU in km
tess = Horizons(id='-95',location='500',epochs=time.jd,id_type='id').vectors(refplane='earth') #state vector
tess_km = tess[['x','y','z','vx','vy','vz']].to_pandas().to_numpy()*au #convert to km
tess_km[:,3:] = tess_km[:,3:]/86400 #convert to km/s
tess_km = tess_km[0] #take the first row

#form the xobs dictionary that is the input for SBIdent location argument
xobs = ','.join([np.format_float_scientific(s,precision=5) for s in tess_km])
xobs_location = {'xobs': xobs}
xobs_location

{'xobs': '1.77056e+05,1.04115e+05,-5.76387e+04,1.52568e-01,1.42229e+00,-4.10623e-02'}

In [10]:
sbid2 = SBIdent(xobs_location, time, center, maglim=19, hwidth=1)

In [11]:
len(sbid2.results)

8

In [12]:
sbid2.results 

Object name,Astrometric RA (hh:mm:ss),"Astrometric Dec (dd mm'ss"")","Dist. from center RA ("")","Dist. from center Dec ("")","Dist. from center Norm ("")",Visual magnitude (V),"RA rate (""/h)","Dec rate (""/h)"
str19,str11,str12,str5,str5,str5,str4,str10,str10
35889 (1999 JA81),00:41:38.47,"-19 04'50.0""",1.E3,3.E3,3591.,18.6,-3.466E+01,-1.338E+01
57855 (2001 XT144),00:40:31.29,"-20 39'25.4""",441.,-2.E3,2406.,18.8,-3.656E+01,-9.510E+00
91101 (1998 HK15),00:37:20.49,"-20 33'53.0""",-2.E3,-2.E3,3028.,17.7,-4.027E+01,-1.105E+00
93867 (2000 WQ115),00:41:38.34,"-19 05'17.7""",1.E3,3.E3,3564.,17.8,-4.067E+01,-1.203E+00
120306 (2004 KE6),00:36:25.74,"-20 30'18.6""",-3.E3,-2.E3,3521.,19.0,-4.064E+01,-2.465E+00
172806 (2004 GT10),00:40:28.79,"-19 44'03.2""",406.,957.,1039.,18.7,-3.013E+01,-1.967E+01
225866 (2001 XM223),00:39:10.24,"-19 52'41.4""",-701.,439.,827.5,19.0,-4.353E+01,4.806E+00
255370 (2005 WS111),00:41:40.48,"-20 05'48.3""",1.E3,-348.,1458.,18.8,-3.333E+01,-5.831E+00


### Request orbital elements instead of RA/DEC

Just like sbid1 but with orbital elements returned instead

In [13]:
sbid3 = SBIdent('567', time, center, maglim=19, hwidth=1, elem=True)

In [14]:
len(sbid3.results)

7

In [15]:
sbid3.results

Object name,Absolute magntiude (H),Magnitude slope (G),Eccentricity,Perihelion (au),Time of perihelion passage (JD),Longitude of ascending node (deg),Argument of perihelion (deg),Inclination (deg),Epoch (JD)
str19,str5,str4,str11,str9,str13,str9,str9,str9,str9
35889 (1999 JA81),14.59,0.15,0.135051263,2.2622576,2459029.70340,107.28924,136.16487,14.314646,2459396.5
57855 (2001 XT144),14.94,0.15,0.145307414,2.2184215,2459106.74290,90.509699,166.42881,14.505108,2459396.5
91101 (1998 HK15),14.82,0.15,0.190003626,2.1612053,2459570.52423,63.944244,326.30676,14.342675,2459396.5
93867 (2000 WQ115),14.53,0.15,0.123655288,2.3321052,2459340.35524,60.323194,260.38542,15.313304,2459396.5
120306 (2004 KE6),15.94,0.15,0.105456059,2.2650747,2459460.78024,67.630093,286.66962,14.095631,2459396.5
172806 (2004 GT10),15.32,0.15,0.152729217,2.4261863,2459419.00451,129.59051,216.02776,16.169861,2459396.5
255370 (2005 WS111),15.07,0.15,0.168087299,2.6063958,2459483.75742,78.532196,283.59020,14.582037,2459396.5


### Use an edges based field of view instead of center based

Define boundaries on a field of view using edges instead of a center + half width.  Use the **skycoord_to_jplstr** class method to translate a SkyCoord RA and DEC to the JPL API format.

In [16]:
edge1 = SBIdent.skycoord_to_jplstr(SkyCoord(10, -20, unit="deg"))
edge2 = SBIdent.skycoord_to_jplstr(SkyCoord(15, -25, unit="deg"))
edges_location = {'fov-ra-lim': "{},{}".format(edge1[0],edge2[0]), 
                'fov-dec-lim': "{},{}".format(edge1[1],edge2[1])}
edges_location

{'fov-ra-lim': '00-40-00.00,01-00-00.00',
 'fov-dec-lim': 'M20-00-00.00,M25-00-00.00'}

In [17]:
sbid4 = SBIdent('567', time, edges_location, maglim=19)

In [18]:
len(sbid4.results)

16

In [19]:
#first 5 results
sbid4.results[0:5]

Object name,Astrometric RA (hh:mm:ss),"Astrometric Dec (dd mm'ss"")","Dist. from center RA ("")","Dist. from center Dec ("")","Dist. from center Norm ("")",Visual magnitude (V),"RA rate (""/h)","Dec rate (""/h)"
str19,str11,str12,str5,str5,str5,str4,str10,str10
6087 Lupo (1988 FK),00:45:38.18,"-20 58'47.7""",-4.E3,5.E3,6577.,17.4,-3.482E+01,-4.127E+01
16668 (1993 XN1),00:47:53.89,"-24 05'12.9""",-2.E3,-6.E3,5971.,17.1,-3.312E+01,-9.151E+00
31074 (1996 WY1),00:46:07.35,"-22 03'08.8""",-3.E3,2.E3,3609.,16.8,-3.792E+01,2.897E+00
33140 (1998 DF3),00:52:50.10,"-23 36'20.0""",2.E3,-4.E3,4621.,17.0,-3.114E+01,-1.319E+01
50158 (2000 AR143),00:53:09.98,"-23 36'59.0""",3.E3,-4.E3,4799.,17.5,-2.560E+01,-3.384E+01
